In [34]:
#importing modules
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string,re,pandas as pd
from sklearn.model_selection import cross_val_score,KFold
from sklearn.feature_extraction.text import CountVectorizer
documents=[]
#reading data
x_data=pd.read_csv("C://Users//neha1//Desktop//training_twitter_x_y_train.csv",delimiter=',')
x_test=pd.read_csv("C://Users//neha1//Desktop//test_twitter_x_test.csv",delimiter=",")
y_train = x_data['airline_sentiment']

# Data Cleaniing

In [35]:
def remove_stop_words(tweets, stops) :
    from nltk import word_tokenize
    stops = list(stopwords.words('english'))
    stops += (list(string.punctuation))
    clean_tweets = []
    for tweet in tweets :
        new_tweet = []
        for word in word_tokenize(tweet):
            if word.lower() not in stops :            
                new_tweet.append(word)
        clean_tweets.append(" ".join(new_tweet))
    return clean_tweets
        

def remove_hrefs(tweets) :
    clean_tweets = []
    for tweet in tweets :
        new_tweet = []
        for word in tweet.split():
            #if not (word.startswith("http://") or word.startswith("https://")):
            if (word.startswith("@")):
                continue
            word = re.sub("[^A-Za-z ]", ' ', word)
            word = re.sub("\s+", " ", word)
            new_tweet.append(word)
            #else:
            #    print("found", word)
        clean_tweets.append(" ".join(new_tweet))
    return clean_tweets


def lower(tweets) :
    clean_tweets = []
    for tweet in tweets :
        clean_tweets.append(tweet.lower())
    return clean_tweets



train_documents = []
test_documents = []
for i in range(len(x_data)):
    train_documents.append(x_data['text'][i])
for i in range(len(x_test)):
    test_documents.append(x_test['text'][i])

#cleaned Data
hrefs_removed = remove_hrefs(train_documents)
stop_words_removed = remove_stop_words(hrefs_removed, stops)
train_docs = lower(stop_words_removed)

hrefs_removed = remove_hrefs(test_documents)
stop_words_removed = remove_stop_words(hrefs_removed, stops)
test_docs  = lower(stop_words_removed)

# Vectorizer

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfid_Vectorizer = TfidfVectorizer(ngram_range=(1,3),max_features=3000,norm='l2',sublinear_tf=True)
X_train_feature = Tfid_Vectorizer.fit_transform(train_docs)
X_test_feature = Tfid_Vectorizer.transform(test_docs)



# Scaling & Sampling

In [37]:
#Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
X_train_feature = scaler.fit_transform(X_train_feature)
X_test_feature = scaler.transform(X_test_feature)

#OverSampling
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE,RandomOverSampler
smt = SMOTE(random_state = 777, k_neighbors = 1)
X_Smote, Y_Smote = smt.fit_sample(X_train_feature, y_train)

# Classifiers

In [48]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 50)
classifier.fit(X_Smote, Y_Smote)
#Cross Validation
print(cross_val_score(classifier,X_Smote,Y_Smote,cv = KFold(3, True, 0)))
print(classifier.score(X_Smote,Y_Smote))
y_pred = classifier.predict(X_test_feature)

In [46]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C = 0.001, solver = 'sag', max_iter=1000, fit_intercept = True)
clf.fit(X_Smote,Y_Smote)
#Cross Validation
print(cross_val_score(classifier,X_Smote,Y_Smote,cv = KFold(3, True, 0)))
print(classifier.score(X_Smote,Y_Smote))
y_pred = classifier.predict(X_test_feature)

C:\Users\neha1\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [47]:
np.savetxt('C:/Users/neha1/Desktop/predictOutput.csv',y_pred,fmt="%s")
